# RAG: Query Transformations

![Query Transformations](./query_transformations.png)

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
os.environ['LANGSMITH_PROJECT']="ai-agents-rags"
os.environ['LANGSMITH_TRACING']="true"

In [5]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

## Multi Query
https://python.langchain.com/docs/how_to/MultiQueryRetriever/#supplying-your-own-prompt
![Multi Query](./multi-query.png)

### Prompt

In [6]:

from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives 
    | ChatOpenAI(temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [7]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

/var/folders/3h/1wlbxc196jgb9hdyjcjcyn500000gn/T/ipykernel_4009/2784938305.py:10: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


7

In [8]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature=0)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition for LLM agents involves breaking down large tasks into smaller, manageable subgoals. This process enables the agent to efficiently handle complex tasks by dividing them into more manageable components.'

In [9]:
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Illustration of how HuggingGPT works. (Image source: Shen et al. 2023)\n\nThe system comprises of 4 stages:\n(1) Task planning: LLM works as the brain and parses the user requests into multiple tasks. There are four attributes associated with each task: task type, ID, dependencies, and arguments. They use few-shot examples to guide LLM to do task parsing and planning.\nInstruction:\n\nThe AI assistant can parse user input to several tasks: [{"task": task, "id", task_id, "dep": dependency_task_ids, "args": {"text": text, "image": URL, "audio": URL, "video": URL}}]. The "dep" field denotes the id of the previous task which generates a new resource that the current task relies on. A special tag "-task_id" refers to the generated text image, audio and video in the dependency task with id as task_id. The task MUST be selected from the following options: {{ Available Task List }}. There is a 

In [11]:
generate_queries

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are an AI language model assistant. Your task is to generate five \ndifferent versions of the given user question to retrieve relevant documents from a vector \ndatabase. By generating multiple perspectives on the user question, your goal is to help\nthe user overcome some of the limitations of the distance-based similarity search. \nProvide these alternative questions separated by newlines. Original question: {question}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x12d9aa480>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x12d9bc110>, root_client=<openai.OpenAI object at 0x12a58cef0>, root_async_client=<openai.AsyncOpenAI object at 0x12d9aa4b0>, tem

In [15]:
#### RETRIEVAL and GENERATION ####
from langchain import hub

# # Prompt
# prompt = ChatPromptTemplate.from_template(template)
# # LLM
# llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# # Post-processing
# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)

# # Chain
# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

# Question
generate_queries.invoke("What is Task Decomposition?")

['1. Can you explain the concept of Task Decomposition?',
 '2. How is Task Decomposition defined in the context of project management?',
 '3. What are the key principles behind Task Decomposition?',
 '4. Could you elaborate on the process of Task Decomposition?',
 '5. In what ways can Task Decomposition be applied in different industries?']

In [16]:
generate_queries.invoke("""
How to optimize the LLM calls in a RAG setup, for example during a conversation - What measures to be taken to remember the past without stacking the converstion?
""")

['1. What strategies can be implemented to improve LLM calls in a RAG setup, especially in the context of conversations?',
 '2. How can one enhance the efficiency of LLM calls within a RAG setup, particularly in conversational scenarios?',
 '3. What steps should be considered to enhance the performance of LLM calls in a RAG setup, specifically in the context of conversations?',
 '4. How can the effectiveness of LLM calls be maximized in a RAG setup, particularly when engaging in conversations?',
 '5. What actions can be taken to optimize LLM calls in a RAG setup, specifically to avoid cluttering conversations with past information?']

In [17]:
generate_queries.invoke("""
How exactly the LLM - Vector store interaction happens during the conversation?
""")

['1. What is the process of interaction between LLM and Vector store during a conversation?',
 '2. Can you explain the mechanism of LLM - Vector store interaction in conversations?',
 '3. How does the interaction between LLM and Vector store occur in conversations?',
 '4. What happens when LLM interacts with Vector store during a conversation?',
 '5. Could you elaborate on the dynamics of LLM - Vector store interaction in conversations?']

In [18]:
generate_queries.invoke("""
Take the case openai chat completion api, we call the api with a specific question. 
Let us say during the 8th interaction of the conversation - context lost the first 5 interactions.
But the 8th interaction should refer the older data. What exactly going on here, Is the chat api makes a call to the vector store?
""")

['1. How does the OpenAI chat completion API handle context loss during interactions, specifically in the 8th interaction where the first 5 interactions are lost?',
 '2. In the scenario of using the OpenAI chat completion API, how does it ensure that the 8th interaction refers to older data when the context of the first 5 interactions is lost?',
 '3. When utilizing the OpenAI chat completion API, how does it manage the situation where context is lost in the conversation, particularly in the 8th interaction needing to reference older data?',
 '4. What mechanisms does the OpenAI chat completion API employ to maintain continuity in conversations when context is lost, such as in the 8th interaction where the first 5 interactions are missing?',
 '5. Does the OpenAI chat completion API make a call to the vector store to handle situations where context is lost, like in the 8th interaction needing to refer back to older data?']

In [19]:
generate_queries.invoke("""
Assume the llm has no access to the vector store, then whats the significance of the vector store?
""")

['1. What is the importance of the vector store if the language model does not have access to it?',
 '2. Why is the vector store significant in the absence of access by the language model?',
 '3. In a scenario where the llm cannot access the vector store, why does the vector store still hold significance?',
 '4. How does the vector store maintain its importance when the llm is unable to access it?',
 '5. What role does the vector store play when the llm has no access to it?']

## RAG Fusion

https://github.com/Raudaschl/rag-fusion

The code accompanying this README is an implementation of RAG-Fusion, a search methodology that aims to bridge the gap between traditional search paradigms and the multifaceted dimensions of human queries. Inspired by the capabilities of Retrieval Augmented Generation (RAG), this project goes a step further by employing multiple query generation and Reciprocal Rank Fusion to re-rank search results. The overarching goal is to move closer to unearthing that elusive 90% of transformative knowledge that often remains hidden behind top search results.
![Rag Fusion](./rag-fusion.png)

In [20]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

#### Reciprocal Rank Fusion (RRF) 
A method to combine multiple ranked lists of documents into a single, more robust ranking.

    RRF is a rank-based ensemble method that boosts documents which appear in multiple results — regardless of their exact position — by giving diminishing credit based on rank.

- It’s often used in RAG pipelines when you have:
- Multiple retrievers (e.g., one keyword-based, one dense embedding).
- Multiple queries (like query rephrasing or expansion).

And you want to merge and rerank all their results into a final top-k list.

score = Σ (1 / (rank_i + k))

- It’s simple, fast, and domain-agnostic.
- It boosts consensus: if a doc is ranked moderately in multiple lists, it may outperform a top-ranked doc from only one list.
- It’s robust against noisy or outlier retrievers.

In [22]:

from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

7

In [24]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

res = final_rag_chain.invoke({"question":question})
res

"Task decomposition for LLM (large language model) agents involves breaking down complex tasks into smaller, manageable subgoals. This process enables the agent to handle complicated tasks more efficiently by transforming them into simpler steps. There are several methods for task decomposition:\n\n1. **Prompting**: Simple prompts can be used, such as asking for the steps to achieve a specific goal or identifying subgoals.\n2. **Task-specific instructions**: Providing specific instructions tailored to the task at hand, like asking for a story outline when writing a novel.\n3. **Human inputs**: Involving human guidance to help define the subgoals and steps necessary for task completion.\n\nTechniques like Chain of Thought (CoT) and Tree of Thoughts (ToT) are commonly employed in this context. CoT encourages the model to think step by step, while ToT explores multiple reasoning possibilities at each step, creating a tree structure of thoughts. This structured approach aids in clarifying 

In [ ]:
lambda x: x.split("\n")

In [25]:
res.split("\n")

['Task decomposition for LLM (large language model) agents involves breaking down complex tasks into smaller, manageable subgoals. This process enables the agent to handle complicated tasks more efficiently by transforming them into simpler steps. There are several methods for task decomposition:',
 '',
 '1. **Prompting**: Simple prompts can be used, such as asking for the steps to achieve a specific goal or identifying subgoals.',
 '2. **Task-specific instructions**: Providing specific instructions tailored to the task at hand, like asking for a story outline when writing a novel.',
 '3. **Human inputs**: Involving human guidance to help define the subgoals and steps necessary for task completion.',
 '',
 "Techniques like Chain of Thought (CoT) and Tree of Thoughts (ToT) are commonly employed in this context. CoT encourages the model to think step by step, while ToT explores multiple reasoning possibilities at each step, creating a tree structure of thoughts. This structured approach 

In [27]:
docs

[(Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Component One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by 

## Query Translation - Decomposition
https://arxiv.org/pdf/2205.10625  
https://arxiv.org/pdf/2212.10509  
https://arxiv.org/pdf/2310.06117  
![Query Translation](./query-translation.png)

In [28]:
from langchain.prompts import ChatPromptTemplate

# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [29]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = ChatOpenAI(temperature=0)

# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

# Run
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke({"question":question})

In [30]:
questions

['1. What is LLM technology and how does it work in autonomous agent systems?',
 '2. What are the specific components that make up an LLM-powered autonomous agent system?',
 '3. How do the main components of an LLM-powered autonomous agent system interact with each other to enable autonomous behavior?']

In [31]:
# Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [32]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

# llm
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

q_a_pairs = ""
for q in questions:
    
    rag_chain = (
    {"context": itemgetter("question") | retriever, 
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")} 
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [33]:
answer

'The main components of an LLM-powered autonomous agent system interact with each other in a coordinated manner to enable autonomous behavior. \n\n1. Planning: The planning component of the system breaks down large tasks into smaller subgoals through subgoal and decomposition. This allows the agent to efficiently handle complex tasks. Additionally, the agent engages in self-reflection and refinement, learning from past actions and refining them for future steps. This iterative process improves the quality of the final results and guides the agent towards achieving its goals.\n\n2. Memory: The memory component of the system consists of short-term memory for in-context learning and long-term memory for retaining and recalling vast amounts of information over extended periods. By leveraging external vector stores for fast retrieval, the agent can access information crucial for decision-making and task execution.\n\n3. Tool use: The tool use component enables the agent to call external API

In [34]:
answer.split('\n')

['The main components of an LLM-powered autonomous agent system interact with each other in a coordinated manner to enable autonomous behavior. ',
 '',
 '1. Planning: The planning component of the system breaks down large tasks into smaller subgoals through subgoal and decomposition. This allows the agent to efficiently handle complex tasks. Additionally, the agent engages in self-reflection and refinement, learning from past actions and refining them for future steps. This iterative process improves the quality of the final results and guides the agent towards achieving its goals.',
 '',
 '2. Memory: The memory component of the system consists of short-term memory for in-context learning and long-term memory for retaining and recalling vast amounts of information over extended periods. By leveraging external vector stores for fast retrieval, the agent can access information crucial for decision-making and task execution.',
 '',
 '3. Tool use: The tool use component enables the agent t

![Agents](./agents.png)

### Answer Individually
![Answer Individually](./query-translation-individually.png)

In [37]:
# Answer each sub-question individually 

# RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""
    
    # Use our decomposition / 
    sub_questions = sub_question_generator_chain.invoke({"question":question})
    
    # Initialize a list to hold RAG chain results
    rag_results = []
    
    for sub_question in sub_questions:
        
        # Retrieve documents for each sub-question
        retrieved_docs = retriever.invoke(sub_question)
        
        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs, 
                                                                "question": sub_question})
        rag_results.append(answer)
    
    return rag_results,sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

In [38]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})

'The main components of an LLM-powered autonomous agent system include the large language model (LLM) as the core controller, planning with subgoal decomposition and reflection, memory with short-term and long-term capabilities, and tool use for accessing external APIs. These components work together to enable the agent to efficiently handle complex tasks, learn from past actions, and access external resources for improved decision-making, ultimately allowing for autonomous behavior.'

In [39]:
answers

["LLM technology refers to large language models used as the core controller in autonomous agent systems. In these systems, LLM functions as the agent's brain, enabling tasks such as planning, subgoal decomposition, reflection, and refinement. LLM can also interact with external tools through APIs to access additional information.",
 'The specific components that make up an LLM-powered autonomous agent system include planning with subgoal decomposition and reflection, memory with short-term and long-term capabilities, and tool use for calling external APIs for additional information. These components work together to enable the agent to efficiently handle complex tasks, learn from past actions, and access external resources for improved decision-making.',
 "In a LLM-powered autonomous agent system, the main components interact as follows: LLM functions as the agent's brain, with planning involving subgoal decomposition and reflection for self-improvement. The agent also utilizes short-

In [40]:
questions

['1. What is LLM technology and how does it work in autonomous agent systems?',
 '2. What are the specific components that make up an LLM-powered autonomous agent system?',
 '3. How do the main components of an LLM-powered autonomous agent system interact with each other to enable autonomous behavior?']

## Query Translation: Step Back
https://arxiv.org/pdf/2310.06117
![Step Back](./step-back.png)

In [42]:
# Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [43]:
generate_queries_step_back = prompt | ChatOpenAI(temperature=0) | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

'What is task decomposition in general?'

In [46]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Response prompt 
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
)

res = chain.invoke({"question": question})

In [47]:
res.split('\n')

['Task decomposition for LLM agents refers to the process of breaking down complex tasks into smaller, more manageable subgoals or steps. This approach allows the agent to efficiently handle intricate tasks by dividing them into simpler components. In the context of LLM-powered autonomous agents, task decomposition is a crucial component of the planning phase, where the agent utilizes large language models to parse user requests and generate multiple tasks with associated attributes such as task type, ID, dependencies, and arguments.',
 '',
 'One common technique used for task decomposition is the Chain of Thought (CoT) method, which instructs the model to "think step by step" and decompose challenging tasks into smaller, more manageable steps. CoT transforms complex tasks into multiple manageable subtasks, shedding light on the model\'s thinking process. Another extension of CoT is the Tree of Thoughts, which explores multiple reasoning possibilities at each step by generating a tree 